In [1]:
# importar librerias
import pandas as pd
import requests
from bs4 import BeautifulSoup


In [2]:
def sacar_url (ciudad, zona, barrio): 
    
    '''    
    A través de la ciudad, la zona y el barrio obtenemos los datos de la pagina de idealista
    
    Revisar el formato que tenga el url en la web, por ej:
    
    url = "https://www.idealista.com/venta-viviendas/sevilla/centro/arenal-museo-tetuan/"
    
    '''
    
    url = f"https://www.idealista.com/venta-viviendas/{ciudad}/{zona}/{barrio}/"
    
    headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36",
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
    "accept-language": "en-US;en;q=0.9",
    "accept-encoding": "gzip, deflate, br"}
    response = requests.get(url, headers=headers)
    
    if (response.status_code == 200):
        
        soup = BeautifulSoup(response.content, "html.parser")
        return soup
        
        
    else: 
        print(f"Error en la solicitud. Código de estado: {response.status_code}")
        
    

In [3]:
def sacar_url_mas_paginas (ciudad, zona, barrio): 
    
    '''    
    A través de la ciudad, la zona y el barrio obtenemos los datos de la pagina de idealista
    
    Revisar el formato que tenga el url en la web, por ej:
    
    url = "https://www.idealista.com/venta-viviendas/sevilla/centro/arenal-museo-tetuan/pagina-2.htm"
    
    '''
    
    url = f"https://www.idealista.com/venta-viviendas/{ciudad}/{zona}/{barrio}/pagina-2.htm"
    
    headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36",
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
    "accept-language": "en-US;en;q=0.9",
    "accept-encoding": "gzip, deflate, br"}
    response = requests.get(url, headers=headers)
    
    if (response.status_code == 200):
        
        soup = BeautifulSoup(response.content, "html.parser")
        return soup
        
        
    else: 
        print(f"Error en la solicitud. Código de estado: {response.status_code}")

In [4]:
def informacion_viviendas(soup): 
    
    # Obtener el recuadro de las viviendas
    
    result = soup.find(class_='items-container items-list')
    anuncios =result.find_all(class_ = "item item_contains_branding item_hightop extended-item item-multimedia-container")
    
    # Lista con cada una de las viviendas
    anuncios_info = result.find_all(class_="item-info-container")
    
    return anuncios_info

In [5]:
def sacar_datos_vivienda(anuncios_info, ciudad, zona, barrio):
    
    lista_viv = []
    
    for anuncio in anuncios_info: 
        nombre = anuncio.find(class_='item-link')['title']
        precio = int(anuncio.find(class_='item-price h2-simulated').text.replace('€', '').replace('.', ''))
        numero_hab = anuncio.find(class_='item-detail-char').find_all(class_='item-detail')[0].text
        metros_cuadrados = anuncio.find(class_='item-detail-char').find_all(class_='item-detail')[1].text
        #descripcion = anuncio.find(class_='item-detail-char').find_all(class_='item-detail')[2].text
        
        lista_viv.append(
            {'Vivienda': nombre, 
            'Precio' : precio, 
            'Numero habitaciones' : numero_hab, 
            'Metros cuadrados': metros_cuadrados, 
            'Ciudad': ciudad, 
            'Zona': zona,
            'Barrio' : barrio 
            #'Descripcion': descripcion
        })
        
    
    df = pd.DataFrame(lista_viv)
        
    return df
        
        

In [6]:
def df_viviendas_zonas(ciudad,zona,barrio):
    
    # Saco la informacion a traves del url
    soup = sacar_url (ciudad, zona, barrio)
    
    #Saco informacion de las viviendas
    
    anuncios_info = informacion_viviendas(soup)
    
    # Creo el data frame 
    
    df = sacar_datos_vivienda(anuncios_info, ciudad, zona, barrio)
    
    try: 
            # Saco la informacion a traves del url
        soup = sacar_url_mas_paginas (ciudad, zona, barrio)
        
        #Saco informacion de las viviendas
        
        anuncios_info = informacion_viviendas(soup)
        
        # Creo el data frame 
        
        df_1 = sacar_datos_vivienda(anuncios_info, ciudad)
        
        df_unido = pd.concat([df, df_1], ignore_index=True)
    
    except Exception as e:
        print(f"Advertencia: No se pudo obtener información de páginas adicionales. Error: {e}")
        # Si ocurre un error, se devuelve solo el primer DataFrame
        df_unido = df
        
        

        
    return df_unido
    

In [7]:
df_sevilla_arenal = df_viviendas_zonas('sevilla', 'centro','arenal-museo-tetuan')

Advertencia: No se pudo obtener información de páginas adicionales. Error: sacar_datos_vivienda() missing 2 required positional arguments: 'zona' and 'barrio'


In [8]:
df_sevilla_arenal

,Vivienda,Precio,Numero habitaciones,Metros cuadrados,Ciudad,Zona,Barrio
0,"Piso en calle General Polavieja, Arenal - Muse...",539000,3 hab.,147 m²,sevilla,centro,arenal-museo-tetuan
1,"Piso en calle O'Donnell, 34, Arenal - Museo - ...",740000,3 hab.,156 m²,sevilla,centro,arenal-museo-tetuan
2,"Piso en calle San Pablo, Arenal - Museo - Tetu...",740000,3 hab.,128 m²,sevilla,centro,arenal-museo-tetuan
3,"Piso en calle San Pablo, Arenal - Museo - Tetu...",760000,3 hab.,148 m²,sevilla,centro,arenal-museo-tetuan
4,"Piso en calle Albareda, Arenal - Museo - Tetuá...",960000,4 hab.,176 m²,sevilla,centro,arenal-museo-tetuan
5,"Piso en calle Cristo del Calvario, Arenal - Mu...",910000,5 hab.,247 m²,sevilla,centro,arenal-museo-tetuan
6,"Piso en calle Méndez Núñez, Arenal - Museo - T...",670000,2 hab.,85 m²,sevilla,centro,arenal-museo-tetuan
7,"Dúplex en calle radio sevilla, Arenal - Museo ...",418000,3 hab.,109 m²,sevilla,centro,arenal-museo-tetuan
8,"Piso en calle Albareda, Arenal - Museo - Tetuá...",540000,3 hab.,125 m²,sevilla,centro,arenal-museo-tetuan
9,"Ático en Arenal - Museo - Tetuán, Sevilla",420000,2 hab.,79 m²,sevilla,centro,arenal-museo-tetuan


In [9]:
def por_barrios (ciudad, zona, barrios): 
    
    lista_df = []
    
    for barrio in barrios: 
        
        df_barrio = df_viviendas_zonas(ciudad,zona,barrio)
        
        lista_df.append(df_barrio)
    
    df_final = pd.concat(lista_df, ignore_index=True)
    
    return df_final
        

In [10]:
barrios = ['arenal-museo-tetuan', 'alameda']

In [11]:
por_barrios('sevilla', 'centro', barrios)

Advertencia: No se pudo obtener información de páginas adicionales. Error: sacar_datos_vivienda() missing 2 required positional arguments: 'zona' and 'barrio'
Advertencia: No se pudo obtener información de páginas adicionales. Error: sacar_datos_vivienda() missing 2 required positional arguments: 'zona' and 'barrio'


,Vivienda,Precio,Numero habitaciones,Metros cuadrados,Ciudad,Zona,Barrio
0,"Piso en calle General Polavieja, Arenal - Muse...",539000,3 hab.,147 m²,sevilla,centro,arenal-museo-tetuan
1,"Piso en calle O'Donnell, 34, Arenal - Museo - ...",740000,3 hab.,156 m²,sevilla,centro,arenal-museo-tetuan
2,"Piso en calle San Pablo, Arenal - Museo - Tetu...",740000,3 hab.,128 m²,sevilla,centro,arenal-museo-tetuan
3,"Piso en calle San Pablo, Arenal - Museo - Tetu...",760000,3 hab.,148 m²,sevilla,centro,arenal-museo-tetuan
4,"Piso en calle Albareda, Arenal - Museo - Tetuá...",960000,4 hab.,176 m²,sevilla,centro,arenal-museo-tetuan
5,"Piso en calle Cristo del Calvario, Arenal - Mu...",910000,5 hab.,247 m²,sevilla,centro,arenal-museo-tetuan
6,"Piso en calle Méndez Núñez, Arenal - Museo - T...",670000,2 hab.,85 m²,sevilla,centro,arenal-museo-tetuan
7,"Dúplex en calle radio sevilla, Arenal - Museo ...",418000,3 hab.,109 m²,sevilla,centro,arenal-museo-tetuan
8,"Piso en calle Albareda, Arenal - Museo - Tetuá...",540000,3 hab.,125 m²,sevilla,centro,arenal-museo-tetuan
9,"Ático en Arenal - Museo - Tetuán, Sevilla",420000,2 hab.,79 m²,sevilla,centro,arenal-museo-tetuan


In [12]:
h = {'hola':'carlota', 1:34}

In [13]:
h.keys()

dict_keys(['hola', 1])

In [14]:
def por_zonas_barrios (ciudad, zonas_barrios):
    
    lista_df = []
    
    for zona, barrios in zonas_barrios.items():
        
        df_zona = por_barrios (ciudad, zona, barrios)
        
        lista_df.append(df_zona)
    
    df_final = pd.concat(lista_df, ignore_index=True)
    
    return df_final

In [15]:
zonas_barrios = {'centro': ['arenal-museo-tetuan', 'alameda'], 
                 'triana': ['calle-betis-pages-del-corro']}

In [16]:
por_zonas_barrios('sevilla', zonas_barrios)

Advertencia: No se pudo obtener información de páginas adicionales. Error: sacar_datos_vivienda() missing 2 required positional arguments: 'zona' and 'barrio'
Advertencia: No se pudo obtener información de páginas adicionales. Error: sacar_datos_vivienda() missing 2 required positional arguments: 'zona' and 'barrio'
Advertencia: No se pudo obtener información de páginas adicionales. Error: sacar_datos_vivienda() missing 2 required positional arguments: 'zona' and 'barrio'


,Vivienda,Precio,Numero habitaciones,Metros cuadrados,Ciudad,Zona,Barrio
0,"Piso en calle General Polavieja, Arenal - Muse...",539000,3 hab.,147 m²,sevilla,centro,arenal-museo-tetuan
1,"Piso en calle O'Donnell, 34, Arenal - Museo - ...",740000,3 hab.,156 m²,sevilla,centro,arenal-museo-tetuan
2,"Piso en calle San Pablo, Arenal - Museo - Tetu...",740000,3 hab.,128 m²,sevilla,centro,arenal-museo-tetuan
3,"Piso en calle San Pablo, Arenal - Museo - Tetu...",760000,3 hab.,148 m²,sevilla,centro,arenal-museo-tetuan
4,"Piso en calle Albareda, Arenal - Museo - Tetuá...",960000,4 hab.,176 m²,sevilla,centro,arenal-museo-tetuan
...,...,...,...,...,...,...,...
85,"Piso en calle Castilla, Calle Betis - Pagés de...",168000,1 hab.,53 m²,sevilla,triana,calle-betis-pages-del-corro
86,"Piso en Calle Betis - Pagés del Corro, Sevilla",225000,1 hab.,55 m²,sevilla,triana,calle-betis-pages-del-corro
87,"Piso en plaza del Altozano, 1, Calle Betis - P...",230000,1 hab.,70 m²,sevilla,triana,calle-betis-pages-del-corro
88,"Ático en calle Pagés del Corro 95, 97, Calle B...",499000,2 hab.,103 m²,sevilla,triana,calle-betis-pages-del-corro


In [17]:
def por_ciudades(ciudades_zonas): 
    
    lista_df_total = []
    
    for ciudad, zonas_barrios in ciudades_zonas.items():
        
        print(f"Extrayendo informacion de la ciudad: {ciudad}")
        
        df_ciudad = por_zonas_barrios(ciudad, zonas_barrios)
        
        lista_df_total.append(df_ciudad)
        
    df_final = pd.concat(lista_df_total, ignore_index=True)
    
    return df_final

In [18]:
ciudades_zonas = {'sevilla': {
    'centro': ['arenal-museo-tetuan', 'alameda'], 
                 'triana': ['calle-betis-pages-del-corro']}, 
    'madrid':{
        'centro': ['palacio', 'sol'],
        'barrio-de-salamanca': ['goya']}
}

In [19]:
df_madrid_sevilla = por_ciudades(ciudades_zonas)

Extrayendo informacion de la ciudad: sevilla
Advertencia: No se pudo obtener información de páginas adicionales. Error: sacar_datos_vivienda() missing 2 required positional arguments: 'zona' and 'barrio'
Advertencia: No se pudo obtener información de páginas adicionales. Error: sacar_datos_vivienda() missing 2 required positional arguments: 'zona' and 'barrio'
Advertencia: No se pudo obtener información de páginas adicionales. Error: sacar_datos_vivienda() missing 2 required positional arguments: 'zona' and 'barrio'
Extrayendo informacion de la ciudad: madrid
Advertencia: No se pudo obtener información de páginas adicionales. Error: sacar_datos_vivienda() missing 2 required positional arguments: 'zona' and 'barrio'
Advertencia: No se pudo obtener información de páginas adicionales. Error: sacar_datos_vivienda() missing 2 required positional arguments: 'zona' and 'barrio'
Advertencia: No se pudo obtener información de páginas adicionales. Error: sacar_datos_vivienda() missing 2 required

In [20]:
df_madrid_sevilla

,Vivienda,Precio,Numero habitaciones,Metros cuadrados,Ciudad,Zona,Barrio
0,"Piso en calle General Polavieja, Arenal - Muse...",539000,3 hab.,147 m²,sevilla,centro,arenal-museo-tetuan
1,"Piso en calle O'Donnell, 34, Arenal - Museo - ...",740000,3 hab.,156 m²,sevilla,centro,arenal-museo-tetuan
2,"Piso en calle San Pablo, Arenal - Museo - Tetu...",740000,3 hab.,128 m²,sevilla,centro,arenal-museo-tetuan
3,"Piso en calle San Pablo, Arenal - Museo - Tetu...",760000,3 hab.,148 m²,sevilla,centro,arenal-museo-tetuan
4,"Piso en calle Albareda, Arenal - Museo - Tetuá...",960000,4 hab.,176 m²,sevilla,centro,arenal-museo-tetuan
...,...,...,...,...,...,...,...
175,"Piso en calle de Don Ramón de la Cruz, Goya, M...",1050000,2 hab.,117 m²,madrid,barrio-de-salamanca,goya
176,"Piso en calle de Alcalá, Goya, Madrid",1690000,3 hab.,165 m²,madrid,barrio-de-salamanca,goya
177,"Piso en calle de Goya, Goya, Madrid",3600000,5 hab.,379 m²,madrid,barrio-de-salamanca,goya
178,"Piso en calle de Ayala, Goya, Madrid",2630000,5 hab.,306 m²,madrid,barrio-de-salamanca,goya


In [21]:
df_madrid_sevilla['Numero habitaciones'].value_counts()

Numero habitaciones
3 hab.     58
2 hab.     49
1 hab.     24
4 hab.     22
5 hab.     18
8 hab.      2
49 m²       1
10 hab.     1
6 hab.      1
53 m²       1
32 m²       1
61 m²       1
54 m²       1
Name: count, dtype: int64

## 20 municipios mas poblados con el barrio mas poblado y sus 5 primeras zonas

In [23]:
municipios_top_20 = ["Madrid", "Barcelona", "Valencia", "Sevilla", "Zaragoza", "Málaga", "Murcia", "Palma", "Las Palmas de Gran Canaria", "Alacant/Alicante","Bilbao", "Córdoba", "Valladolid", "Vigo","L´Hospitalet de Llobregat","Gijón", "Vitoria-Gasteiz", "A Coruña", "Elche/Elx", "Granada"]

In [24]:
barrios_poblados = {
    "madrid": {"carabanchel": ['san-isidro', 'puerta-bonita', 'vista-alegre','buena-vista' 'abrantes']},
    "barcelona": {"eixample":['la-dreta-de-l-eixample','l-antiga-esquerra-de-l-eixample', 'la-nova-esquerra-de-l-eixample', 'la-sagrada-familia','sant-antoni']},
    "valencia": {'quatre-carreres': ['mont-olivet','malilla','en-corts', 'ciutat-de-les-arts-i-de-les-ciencies', 'la-punta']},
    "sevilla": {'sevilla-este': ['avenida-de-las-ciencias','alcalde-l-urunuela-palacio-de-congresos',  'emilio-lemos'], 
                'parque-alcosa': ['parque-alcosa'], 
                'torreblanca': ['torreblanca']},
    "zaragoza": {'delicias': ['barrio-del-ave','ciudad-jardin-parque-roma', 'mercado-san-valero', 'delicias', 'parque-delicias-la-bozada']},
    "malaga":{'centro': ['la-trinidad', 'centro-historico', 'perchel-sur-el-bulto', 'el-ejido']},
    "murcia": {'sur': ['el-carmen', 'la-purisima-barriomar']},
    "palma": {'las-avenidas': ['pere-garau']},
    "Las Palmas de Gran Canaria": ["Triana", "Vegueta", "Las Canteras"],
    "Alicante": ["Centro", "San Blas", "Albufereta"],
    "Bilbao": ["Abando", "Deusto", "Bilbo Zaharra"],
    "Córdoba": ["Centro", "Noroeste", "Sur"],
    "Valladolid": ["Centro", "Parquesol", "La Victoria"],
    "Vigo": ["Centro", "Teis", "Bouzas"],
    "L'Hospitalet de Llobregat": ["Centre", "Collblanc", "Santa Eulàlia"],
    "Gijón": ["Centro", "Viesques", "El Llano"],
    "Vitoria-Gasteiz": ["Centro", "Adurtzabal", "San Martín"],
    "A Coruña": ["Centro", "Ensanche", "Ronda de Outeiro"],
    "Elche/Elx": ["Centro", "Altabix", "Torreta"],
    "Granada": ["Centro", "Ronda", "Zaidín"],
}